In [136]:
import pandas as pd
import utils
import plotly.graph_objects as go
import instrument

In [159]:
# Plot candle price
pair ="GBP_JPY"
granularity = "H1"
ma_list = [9,21]
i_pair = instrument.Instrument.get_instrument_by_name(pair)

In [160]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))
non_cols = ['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [161]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()
for ma in ma_list:
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(drop=True, inplace=True)



In [162]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64
0,2023-01-20T05:00:00.000000000Z,159.630,159.976,159.582,159.906,159.207313,158.830109
1,2023-01-20T06:00:00.000000000Z,159.906,159.910,159.694,159.744,159.281937,158.862391
2,2023-01-20T07:00:00.000000000Z,159.741,159.741,159.204,159.352,159.318688,158.893609
3,2023-01-20T08:00:00.000000000Z,159.352,159.716,159.240,159.650,159.367688,158.932203
4,2023-01-20T09:00:00.000000000Z,159.646,159.878,159.586,159.780,159.412188,158.971641


In [163]:
def is_trade(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return 1 #Buy
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return -1 #Sell
    return 0

In [164]:
df_ma['DIFF'] = df_ma.MA_16 - df_ma.MA_64
df_ma['DIFF_PREV']= df_ma.DIFF.shift(1)
df_ma['IS_TRADE']= df_ma.apply(is_trade, axis=1)
df_trades = df_ma[df_ma.IS_TRADE!=0].copy()


In [168]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE,DELTA,GAIN
60,2023-01-24T17:00:00.000000000Z,160.281,160.450,160.152,160.265,160.746562,160.806797,-0.060234,0.019875,-1,101.1,-101.1
105,2023-01-26T14:00:00.000000000Z,160.954,161.467,160.878,161.276,160.628500,160.615781,0.012719,-0.035875,1,-55.2,-55.2
170,2023-01-31T07:00:00.000000000Z,160.813,160.924,160.720,160.724,160.999125,161.002063,-0.002938,0.028344,-1,-174.1,174.1
262,2023-02-06T03:00:00.000000000Z,159.174,159.236,158.889,158.983,158.439875,158.349422,0.090453,-0.024031,1,-119.5,-119.5
300,2023-02-07T17:00:00.000000000Z,157.802,157.872,157.614,157.788,158.530813,158.554047,-0.023234,0.088922,-1,96.4,-96.4


In [166]:
df_trades['DELTA']= (df_trades.mid_c.diff()/i_pair.pipLocation).shift(-1)  # we want them in pips
df_trades['GAIN']= df_trades['DELTA'] * df_trades['IS_TRADE'] # calculate the actual gain in our trades, negative means to sell

In [167]:
df_trades['GAIN'].sum()

-990.2999999999878

In [122]:
df_plot = df_ma.iloc[80:120]

In [123]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C'
))
for ma in ma_list:
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time, 
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col
        ))


fig.update_layout(width=1000,height=600,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()
